In [1]:
!conda create --name nilmtk python=3.8 -y

Retrieving notices: ...working... done
Channels:
 - rapidsai
 - nvidia
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/nilmtk

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            2_gnu          23 KB  conda-forge
    bzip2-1.0.8                |       hd590300_5         248 KB  conda-forge
    libgcc-ng-13.2.0           |       h807b86a_3         755 KB  conda-forge
    libgomp-13.2.0             |       h807b86a_3         412 KB  conda-forge
    libnsl-2.0.1               |       hd590300_0          33 KB  conda-forge
    libxcrypt-4.4.36           |       hd590300_1          98 KB  conda-forge
    ncurses-6.4                |       h59595ed_2     

In [2]:
!conda clean -ya

Will remove 12 (27.1 MB) tarball(s).
Will remove 1 index cache(s).
Will remove 11 (426 KB) package(s).
There are no tempfile(s) to remove.
There are no logfile(s) to remove.


In [7]:
import os

old_path = os.environ["PATH"]
new_path = f"/opt/conda/envs/nilmtk/bin:{old_path}"

%env PATH=$new_path

env: PATH=/opt/conda/envs/nilmtk/bin:/opt/conda/envs/nilmtk/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


In [8]:
!conda config --add channels conda-forge

In [9]:
!conda install -c nilmtk nilmtk

Channels:
 - nilmtk
 - conda-forge
 - rapidsai
 - nvidia
 - defaults
 - file:///tmp/conda
Platform: linux-64

UnavailableInvalidChannel: HTTP 404 UNAVAILABLE OR INVALID for channel conda <file:///tmp/conda>

The channel is not accessible or is invalid.

You will need to adjust your conda configuration to proceed.
Use `conda config --show channels` to view your configuration's current state,
and use `conda config --show-sources` to view config file locations.

As of conda 4.3, a valid channel must contain a `noarch/repodata.json` and
associated `noarch/repodata.json.bz2` file, even if `noarch/repodata.json` is
empty. Use `conda index /tmp/conda`, or create `noarch/repodata.json`
and associated `noarch/repodata.json.bz2`.




In [12]:
!pip install git+https://github.com/nilmtk/nilmtk

  Cloning https://github.com/nilmtk/nilmtk to /tmp/pip-req-build-65f03w1b
  Running command git clone --filter=blob:none --quiet https://github.com/nilmtk/nilmtk /tmp/pip-req-build-65f03w1b
  Resolved https://github.com/nilmtk/nilmtk to commit b2c514479cef478cab872cb635056da08d5352a1
  Preparing metadata (setup.py) ... done


In [13]:
!pip install git+https://github.com/nilmtk/nilm_metadata

  Cloning https://github.com/nilmtk/nilm_metadata to /tmp/pip-req-build-f1ax2m8y
  Running command git clone --filter=blob:none --quiet https://github.com/nilmtk/nilm_metadata /tmp/pip-req-build-f1ax2m8y
  Resolved https://github.com/nilmtk/nilm_metadata to commit 7ed4bab9062d04cb35c6b6000b451715dc5ab4af
  Preparing metadata (setup.py) ... done
  Created wheel for nilm-metadata: filename=nilm_metadata-0.2.5-py3-none-any.whl size=24338 sha256=a22b3104e876ffd84feae59bfaf1208ff6b00681ec11cdccb5a6d192060d4552
  Stored in directory: /tmp/pip-ephem-wheel-cache-kbii0utq/wheels/a5/b5/66/9fa2522471f4bfc64708ad437946a36c168178dc021bc05f6b
Successfully built nilm-metadata


In [17]:
!ls ../input/greend

GREEND_0-1_311014


In [14]:
!rm -rf greend
!mkdir greend
!cp -r ../input/greend/GREEND_0-1_311014/building0 greend/
!cp -r ../input/greend/GREEND_0-1_311014/building1 greend/
!cp -r ../input/greend/GREEND_0-1_311014/building2 greend/
!cp -r ../input/greend/GREEND_0-1_311014/building3 greend/
!cp -r ../input/greend/GREEND_0-1_311014/building4 greend/

In [55]:
!rm -rf greend/

In [15]:
!mkdir activations

In [69]:
!rm -r activations/dish_washer

In [73]:
%%writefile main.py

from nilmtk import DataSet
import numpy as np
import os
from nilmtk.dataset_converters.greend.convert_greend import convert_greend

#convert_greend("greend", "greend.h5")

greend = DataSet("greend.h5")
activations = greend.buildings[5].elec["dish washer"].get_activations(min_on_duration=90, min_off_duration=90, on_power_threshold=30)

lens = [len(a) for a in activations]
lens_mean = np.mean(lens)
print(len(activations), lens_mean)

os.mkdir(f"activations/dish_washer")

for i, activ in enumerate(activations):
  activ.reset_index(drop=True, inplace=True)
  activ.dropna(inplace=True)
        
  activ.to_csv(f"activations/dish_washer/{i}.csv", index=False)

print("OK")

Overwriting main.py


In [74]:
!/opt/conda/envs/nilmtk/bin/python ./main.py

727 1667.0013755158184
./main.py:22: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  activ.to_csv(f"activations/dish_washer/{i}.csv", index=False)
OK
Closing remaining open files:/tmp/nilmtk-i76na_qp.h5...donegreend.h5...done


In [75]:
import shutil
shutil.make_archive("activs", 'zip', "activations")

'/kaggle/working/activs.zip'